Let's generate a cryptosystem with random private - public key pair

Then encrypt vector embeddings of facial images with this cryptosystem

In [3]:
# built-in dependencies
import os

# 3rd party dependencies
from deepface import DeepFace
import tenseal as ts
from tqdm import tqdm

# project dependencies
import utils

### Build the cryptosystem

<img src="figures/fhe-security-levels.webp"/>

<img src="figures/symmetric-and-public-key-security.webp"/>

In [4]:
# provides 128-bit security
context = ts.context(
    ts.SCHEME_TYPE.CKKS,
    poly_modulus_degree = 8192, # n = 8192
    coeff_mod_bit_sizes = [60, 40, 40, 60] # q = 60+40+40+60 = 200
)

# smaller q values result in stronger encryption
# the first and last bit sizes (60 bits here) should always be the same.
# The middle items (40 bits each in this case) should also be uniform for optimal performance.

context.generate_galois_keys()
context.global_scale = 2**40

# alternative configuration for 128-bit security
# context = ts.context(
#     ts.SCHEME_TYPE.CKKS,
#     poly_modulus_degree = 16384,
#     coeff_mod_bit_sizes = [31, 60, 60, 60, 60, 60, 60, 31] # q = 31+60+60+60+60+60+60+31 = 422
# )
# context.generate_galois_keys()
# context.global_scale = 2**60

In [5]:
secret_context = context.serialize(save_secret_key = True)
utils.write_data("secret.txt", secret_context)

In [6]:
context.make_context_public() #drop the secret_key from the context
public_context = context.serialize()
utils.write_data("public.txt", public_context)

In [7]:
del context, secret_context, public_context

### Facial Database

In [8]:
# https://github.com/serengil/deepface/tree/master/tests/dataset
database = {
    "angelina": "../../deepface/tests/dataset/img1.jpg",
    "jennifer": "../../deepface/tests/dataset/img3.jpg",
    "scarlett": "../../deepface/tests/dataset/img9.jpg",
    "mark": "../../deepface/tests/dataset/img13.jpg",
    "jack": "../../deepface/tests/dataset/img16.jpg",
    "elon": "../../deepface/tests/dataset/img18.jpg",
    "jeff": "../../deepface/tests/dataset/img20.jpg",
    "marissa": "../../deepface/tests/dataset/img22.jpg",
    "matt": "../../deepface/tests/dataset/img29.jpg",
    "leonardo": "../../deepface/tests/dataset/img35.jpg",
    "george": "../../deepface/tests/dataset/img41.jpg",
    "katy": "../../deepface/tests/dataset/img42.jpg",

}

### Find Embeddings and Encrypt

In [9]:
context = ts.context_from(utils.read_data("secret.txt"))

In [10]:
for identity, img_path in tqdm(database.items()):
    embedding = DeepFace.represent(img_path=img_path, model_name="Facenet")[0]["embedding"]
    encrypted_embedding = ts.ckks_vector(context, embedding)
    encrypted_embedding_proto = encrypted_embedding.serialize()

    utils.write_data(f"database/{identity}.txt", encrypted_embedding_proto)

100%|██████████| 12/12 [00:09<00:00,  1.29it/s]
